In [1]:
import time

In [55]:
import numpy as np
from datetime import datetime, timedelta

def get_features_from_file(a_member_id):
    file_handler = open('%s.txt' % a_member_id, 'r', encoding = 'utf-8')
    lines = file_handler.readlines()
    file_handler.close()


    

    """
    first part 
    """
    def change_date(previous_time, time):
        nonlocal Change_day_Flag #控制 Flag 用
        
        if time.date() != previous_time.date():
            Change_day_Flag = True
            return True
        else:
            return False
        
    def change_session(previous_session, session, previous_time, time, previous_source, source):
        if ( time - previous_time >  timedelta(seconds = 1800) )  or source != previous_source : 
            return True
        elif  time.date() == previous_time.date() and session != previous_session :
            return True
        else:
            return False

    def change_session_series(previous_session, session, previous_time, time, previous_source, source):
        nonlocal Change_day_Flag #控制 Flag 用
        
        if Change_day_Flag == True:
            if change_session(previous_session, session, previous_time, time, previous_source, source):
                Change_day_Flag = False
                return True
        else:
            return False
        
    def is_offline(behavior):
        if behavior == 'Offline':
            return True
        else:
            return False
            
            
            
    """
    second part
    """
    def is_buying(behavior): #看是不是購買行為
        if behavior in ['Purchase', 'Offline']:
            return True
        else:
            return False
        
    def is_salepage_or_cart_or_fav(behavior): ##看是不是salepage 或者 購物車 或者 加入最愛行為
        if behavior in ['ViewSalePage', 'Cart', 'Fav']:
            return True
        else:
            return False
        
        
    def update_product_dictionary(product_id, behavior, time = ''): 
        #把瀏覽的商品ID給記錄下來，每個id對應5個元素，[view, cart, fav, buy, (last_time, last_type)]
        nonlocal product_dictionary
        
        if product_id not in product_dictionary:
            product_dictionary[product_id] = [0,0,0,0,('', '')]

        if behavior in ['ViewSalePage', 'Cart', 'Fav']:
            for i in range(3):
                if behavior == ['ViewSalePage', 'Cart', 'Fav'][i]:
                    product_dictionary[product_id][i] +=1
                    break
            product_dictionary[product_id][4] = (time, behavior)
            
        elif behavior in ['Purchase', 'Offline']:
            product_dictionary[product_id][3] +=1
                

        
    def update_purchase_dictionary(product_id, Status):
        nonlocal purchase_dictionary
        if Status == 'Fin':
            if product_id not in purchase_dictionary['finish_id'] :
                purchase_dictionary['finish_id'].append(product_id)
        

        elif Status == 'Ret':
            if product_id not in purchase_dictionary['return_id'] :
                purchase_dictionary['return_id'].append(product_id)
            
    def update_purchase_time(time, amount, discount, where): #第一筆購買資料出現後 要更新該筆購物的時間地點金額
        nonlocal purchase_dictionary
        

        purchase_dictionary['amount'] = amount
        purchase_dictionary['time'] = time
        purchase_dictionary['discount'] = discount
        purchase_dictionary['where'] = 'Online' if where=='Purchase' else 'Offline'

    def is_first_purchase_row(): #看該筆購買行為是不是一個購買行為的第一次出現購買紀錄
        nonlocal Next_purchase_Flag
        if Next_purchase_Flag == False: #將會跟新一個flag，用來知道是否要更新
            Next_purchase_Flag = True
            return True
        else:
            return False
        
    def is_same_purchase(time):  #看該筆購買行為是不是同一個單號的東西
        if  time == purchase_dictionary['time']:
            return True
        else:
            return False
        
    def needs_save_everything(time = '', behavior = ''):#看是不是需要更新
        nonlocal Next_purchase_Flag, tem_buying_behaviour, purchase_dictionary
        

        
        if Next_purchase_Flag == True :#如果有之前有購買紀錄出現
        
            if time != purchase_dictionary['time'] or behavior not in ['Purchase', 'Offline']: #看看是否是連續的購買行為
                
                #如果是的話，看看這個購買行為有沒有意義(是不是有status是finish)
                is_valid_purchase = False
                for a_id,a_status in tem_buying_behaviour:
                    if a_status =='Fin':
                        is_valid_purchase = True
                        break
                
                
                
                if is_valid_purchase: #如果是有效的購買行為 將會return True 並且之後會呼叫save_everything()
                    Next_purchase_Flag = False
                    return True
                
                else: #不然的話就把 tem_buying_behaviour，Next_purchase_Flag 給歸零
                    tem_buying_behaviour = []
                    Next_purchase_Flag = False
                    return False

        return False
        
    def save_everything(): #更新全部的東西
        nonlocal purchase_dict_records, product_dictionary_records, purchase_dictionary, product_dictionary, tem_buying_behaviour
        
        
        for a_id,a_status in tem_buying_behaviour:
            update_purchase_dictionary(a_id,a_status)
            update_product_dictionary(a_id, 'Purchase')
        
        purchase_dict_records.append(purchase_dictionary)
        product_dictionary_records.append(product_dictionary)
        
        # reset those value
        purchase_dictionary = dict() 
        purchase_dictionary['finish_id'] = []
        purchase_dictionary['return_id'] = []
        product_dictionary = dict()
        tem_buying_behaviour = [] 
        
    '''
    third part
    '''
        
        
    def update_F_M(purchase_time, purchase_amount):
        nonlocal F_list, M_list
        
        
        if purchase_time.year == 2018:
            i = purchase_time.month - 3
        elif purchase_time.year == 2019:
            i = purchase_time.month + 9
        
        F_list[i] +=1
        M_list[i] += purchase_amount
        
        
    def update_conversion_list(view, cart, fav, product_id):
        nonlocal conversion_list
        
        if view !=0:
            conversion_list[0].add(product_id)
        if cart !=0:
            conversion_list[1].add(product_id)
        if fav !=0:
            conversion_list[2].add(product_id)
            
    def update_conversion_metrics(conversion_list,  purchase_dict):
        nonlocal conversion_metrics_all
        
        def conversion_rate(buy_set, other_set):
            if len(other_set) == 0:
                return np.nan
            else:
                return round( len(  buy_set.intersection(other_set)   ) /  len(other_set) , 4)
            
        buy_set = set( purchase_dict['finish_id'] )
        view_set, cart_set, fav_set = conversion_list
        
        
        v_c = conversion_rate(buy_set, view_set)
        c_c = conversion_rate(buy_set, cart_set)
        f_c = conversion_rate(buy_set, fav_set)
        
        conversion_metrics_all.append(  [v_c, c_c, f_c ])
        
    def update_conversion_rate_list(conversion_metrics, to_be_added_list):
        for i in range(3):
            to_be_added_list[i].append(conversion_metrics[i])
            
            

            
    '''
    part of variables
    '''

    Change_day_Flag = False #代表是否有曾經換過日期
    # 紀錄product dictionary
    purchase_dict_records = [] #放dict, purchase_dictionary
    product_dictionary_records = [] #放dict，紀錄product_dict

    purchase_dictionary = dict() #紀錄每個purchase的狀態 ，{amount:總金額, discount:折扣金額, finish_id:[], return_id:[], time:時間, where:地點}
    purchase_dictionary['finish_id'] = []
    purchase_dictionary['return_id'] = []


    product_dictionary = dict() #紀錄每個product_id的狀態，{ id : (看次數、購物車次數、FAV次數、買次數, (最後一次使用的時間與type)) } 


    tem_buying_behaviour = [] #紀錄遇到purchase後 的Status以及productID狀態，為了避免concel or fail or 只有退貨的狀況






    '''
    part of main function
    '''
    Flag_of_first = True
    Next_purchase_Flag = False

    for i in range(len(lines)):
        line = lines[i]
        time, ad, session, behavior, source, number = line.strip('\n').split(',')
        time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S') 
        
        
        '''
        處理商品購買，與購物週期
        '''
        
        if needs_save_everything(time, behavior): #看需不需要儲存購買行為了!
            save_everything()#存東西
            
        if is_salepage_or_cart_or_fav(behavior): #看是不是salepage 或者 購物車 或者 加入最愛行為
            update_product_dictionary(number, behavior, time) #把瀏覽的商品ID給記錄下來
                
            
            
            
        elif is_buying(behavior): #看是不是購買行為
            
            # try, except 是為了處理有些線下行為不知道對應的線上商品ID
            try:
                product_id, amount, discount, Status = number.split()
            except:
                amount, discount, Status = number.split()
                product_id = ''
            
            if is_first_purchase_row(): #看該筆購買行為是不是一個購買行為的第一次出現購買紀錄
                
                update_purchase_time(time, amount, discount, behavior)  #第一筆才要記錄
                tem_buying_behaviour.append( (product_id, Status) )
            
            
            elif is_same_purchase(time):
                tem_buying_behaviour.append( (product_id, Status) )    
        
        
        
        ''' 
        處理session 連續
        '''
        if Flag_of_first and not is_offline(behavior):
            previous_time = time #上一筆的時間
            previous_session = session #上一筆的session
            previous_source = source #上一筆的source
            date_list = [previous_time.date()]
            session_series_list = []
            session_list = []
            session_first_data_list =[line.split(',')]
            tem_session_list = [previous_time]
            tem_session_series_list = [previous_time]
            Flag_of_first = False
            continue
            
        if is_offline(behavior):
            continue
        if change_date(previous_time, time): #如果換天
            date_list.append(time.date())
        
        # session轉換
        if change_session(previous_session, session, previous_time, time, previous_source, source) :#如果換session
            session_list.append(tem_session_list) #GISH
            tem_session_list = [time] #GISH
            session_first_data_list.append(line.split(',')) #G 
        else: 
            tem_session_list.append(time) #G
        # session series轉換    
        if change_session_series(previous_session, session, previous_time, time, previous_source, source):
            session_series_list.append(tem_session_series_list)
            tem_session_series_list = [time]   
        else:
            tem_session_series_list.append(time)
            
        #記得更新上一本的資料
        previous_time = time #上一筆的time
        previous_session = session #上一筆的session
        previous_source = source #上一筆的source
       
       

    # 怕最後一筆資料是購買資料:
    if needs_save_everything():
        save_everything()
        

    session_series_list.append(tem_session_series_list)
    session_list.append(tem_session_list)
        

        
        
    '''
    part 5 => calculate important features
    '''

    # session viewing time avg.
    session_viewtime_list = [] # 此為儲存每個session的瀏覽時間，如果只有一筆資料就為 None
    session_avg_viewtime_list = []

    for session_times in session_list:
        view_time = session_times[-1]-session_times[0]
        session_viewtime_list.append(view_time.seconds +  view_time.microseconds/1000000 )
        if len(session_times) >1:
            average_view_time = view_time / (len(session_times)-1)
            session_avg_viewtime_list.append(   average_view_time.seconds +  average_view_time.microseconds/1000000    )

        else:
            session_avg_viewtime_list.append(np.nan)

    # F and M
    F_list = [0]*14
    M_list = [0]*14
    total_discount_amount = 0
    for a_record in purchase_dict_records:
        purchase_time = a_record['time']
        purchase_amount = int( a_record['amount'] )
        update_F_M(purchase_time, purchase_amount)
        total_discount_amount -= int( a_record['discount'] )
                
                
    # conversion
    conversion_metrics_all = [] #裡面放每一次交易的轉換率list = [view 轉換率, cart 轉換率, fav轉換率]

    for i in range(len(product_dictionary_records)):
        a_dict = product_dictionary_records[i]
        conversion_list = [set(), set(), set()]
        
        for product_id in a_dict.keys():
            view, cart, fav, buy, (time, last) = a_dict[product_id]
            update_conversion_list(view, cart, fav, product_id)
            
        update_conversion_metrics(conversion_list, purchase_dict_records[i])
        
        
        
    offline_conversion_rate_list = [[],[],[]]
    online_conversion_rate_list = [[],[],[]]
    total_conversion_rate_list = [[],[],[]]
    for i in range(len(conversion_metrics_all)):
        if purchase_dict_records[i]['where'] == 'Offline':
            update_conversion_rate_list(conversion_metrics_all[i], offline_conversion_rate_list)
        else:
            update_conversion_rate_list(conversion_metrics_all[i], online_conversion_rate_list)
        update_conversion_rate_list(conversion_metrics_all[i], total_conversion_rate_list)
        
        

    # activity
    online_number = 0
    for a_dict in purchase_dict_records:
        if a_dict['where'] == 'Online':
            online_number +=  1 

    number_of_online_behavior = 0
    for a_list in session_list:
        number_of_online_behavior += len(a_list)        
        
    '''
    part 6 => output those features to csv file
    '''
    # conversion rate
    total_conversion_rate = len(purchase_dict_records) / len(session_list)
    off_view_c = np.nanmean(offline_conversion_rate_list[0])
    off_cart_c = np.nanmean(offline_conversion_rate_list[1])
    off_fav_c = np.nanmean(offline_conversion_rate_list[2])
    on_view_c = np.nanmean(online_conversion_rate_list[0])
    on_cart_c = np.nanmean(online_conversion_rate_list[1])
    on_fav_c = np.nanmean(online_conversion_rate_list[2])
    total_view_c = np.nanmean(total_conversion_rate_list[0])
    total_cart_c = np.nanmean(total_conversion_rate_list[1])
    total_fav_c = np.nanmean(total_conversion_rate_list[2])



    # online ratio
    online_ratio = online_number / len(purchase_dict_records) if len(purchase_dict_records)>0 else np.nan

    # activity ratio
    total_using_percentage = len(session_series_list) / ((datetime(2019,4,30).date() - session_list[0][0].date()).days + 1)

    actually_using_percentage = len(session_series_list) / ((session_list[-1][-1].date() - session_list[0][0].date()).days+1)

    view_per_date = number_of_online_behavior / len(session_series_list)

    # ave_viewing time

    viewtime_ave, view_time_med = np.nanmean(session_avg_viewtime_list), np.nanmedian(session_avg_viewtime_list)


    # M and F amount 201803 ~ 201904


    a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z, z1, z2 = F_list + M_list 
    total_F,total_M = np.sum(F_list), np.sum(M_list)
    total_discount_percentage = np.nan if total_M == 0 else  (total_discount_amount / ( total_M + total_discount_amount))
    
    return online_ratio, total_using_percentage , actually_using_percentage, view_per_date, total_conversion_rate, off_cart_c, off_fav_c, off_view_c, on_cart_c, on_fav_c, on_view_c, total_cart_c, total_fav_c, total_view_c, viewtime_ave, view_time_med, a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z, z1, z2, total_F,total_M, total_discount_percentage


# debug用

In [68]:
def debug(a_id):

    file_handler = open('%s.txt' % a_id, 'r', encoding = 'utf-8')
    lines = file_handler.readlines()
    file_handler.close()

    def is_buying(behavior): #看是不是購買行為
        if behavior in ['Purchase', 'Offline']:
            return True
        else:
            return False
 
    for i in range(len(lines)):
        line = lines[i]
        time, ad, session, behavior, source, number = line.strip('\n').split(',')

            
        if is_buying(behavior): #看是不是購買行為
            
            # try, except 是為了處理有些線下行為不知道對應的線上商品ID
            try:
                product_id, amount, discount, Status = number.split()
            except:
                amount, discount, Status = number.split()
                

In [69]:

for a_id in member_id_list:
    try:
        debug(r'.\DataForNTU\merge\%s' %a_id)
    except:
        print(a_id)

# all member ID 開始輸出

In [70]:
member_id_list = []
with open(r'.\DataForNTU\memberID\all_ID_member.txt', 'r') as f:
    member_id_list  = [x.strip('\n') for x in f.readlines()]

In [71]:
len( member_id_list )

76401

In [72]:
f_write = open('91app_features.txt', 'w')
f_write.write('memberid,online_ratio,total_day_using_percentage,actually_using_percentage,view_per_date,total_conversion_rate,off_cart_c,off_fav_c,off_view_c,on_cart_c,on_fav_c,on_view_c,total_cart_c,total_fav_c,total_view_c,viewtime_ave,view_time_med,201803F,201804F,201805F,201806F,201807F,201808F,201809F,201810F,201811F,201812F,201901F,201902F,201903F,201904F,201803M,201804M,201805M,201806M,201807M,201808M,201809M,201810M,201811M,201812M,201901M,201902M,201903M,201904M,total_F,total_M,total_discount_percentage\n')

t1 = time.time()
cnt = 0
for i in member_id_list:
    cnt+=1
    
    if cnt%500 ==0:
        print(cnt, '/76401, time=', time.time()-t1)
    

    values = get_features_from_file(r'.\DataForNTU\merge\%s' %i)

        

    a_line = i +','+  ','.join([str(x) for x in values]) + '\n'
    f_write.write(a_line)
    
print(time.time()-t1)
f_write.close()

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:413: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:414: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:415: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:416: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:417: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:418: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:419: RuntimeWarning: Mean of empty slice
c:\users\user\appdata\local\programs\python\python37\lib\site-

500 /76401, time= 1.6089904308319092
1000 /76401, time= 3.374201774597168
1500 /76401, time= 5.01444149017334
2000 /76401, time= 6.685924053192139
2500 /76401, time= 8.48237681388855
3000 /76401, time= 10.153832912445068
3500 /76401, time= 11.35670256614685
4000 /76401, time= 12.49703335762024
4500 /76401, time= 13.371827602386475
5000 /76401, time= 15.293277502059937
5500 /76401, time= 16.91789674758911
6000 /76401, time= 18.71435022354126
6500 /76401, time= 20.479572057724
7000 /76401, time= 22.182286977767944
7500 /76401, time= 24.134952545166016
8000 /76401, time= 25.915758848190308
8500 /76401, time= 27.55599856376648
9000 /76401, time= 28.977566480636597
9500 /76401, time= 29.930452823638916
10000 /76401, time= 31.055203199386597
10500 /76401, time= 32.86727833747864
11000 /76401, time= 34.694947957992554
11500 /76401, time= 36.31959295272827
12000 /76401, time= 38.084776878356934
12500 /76401, time= 39.80315017700195
13000 /76401, time= 41.740195989608765
13500 /76401, time= 43.

# 讀取看看

In [73]:
import pandas as pd
f_write.close()
pd.read_csv('91app_features.txt')

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,memberid,online_ratio,total_day_using_percentage,actually_using_percentage,view_per_date,total_conversion_rate,off_cart_c,off_fav_c,off_view_c,on_cart_c,...,201810M,201811M,201812M,201901M,201902M,201903M,201904M,total_F,total_M,total_discount_percentage
0,23722,NaN,0.002849,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
1,44751,NaN,0.022923,0.026846,3.875000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
2,66732,NaN,0.027160,0.108911,4.363636,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
3,108059,NaN,0.003058,1.000000,2.000000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
4,257559,NaN,0.006061,0.250000,1.500000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
5,314792,NaN,0.006116,0.016529,1.000000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
6,394660,NaN,0.012019,0.016129,1.600000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
7,437828,NaN,0.002770,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
8,439730,NaN,0.002817,1.000000,3.000000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
9,519517,NaN,0.002488,1.000000,4.000000,0.000000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,NaN
